In [1]:
import pandas as pd
data = pd.read_csv("./data.csv")
data.columns = ["URL", "Text", "Start-Indices", "End-Indices", "Text-Type"]
#data["Text-Type"] = 5
data.head()

,URL,Text,Start-Indices,End-Indices,Text-Type
0,https://stackoverflow.com/questions/37958781,So does that mean it is better than the defaul...,[],[],0
1,https://stackoverflow.com/questions/37958781,It is fundamentally a heuristic based approach...,[],[],0
2,https://stackoverflow.com/questions/37958781,Calling it a heuristic approach is not meant t...,[],[],0
3,https://stackoverflow.com/questions/37958781,"The text in question was Moby Dick, and the od...",[],[],0
4,https://stackoverflow.com/questions/37958783,A table containing only debit and credit colum...,[],[],0


In [2]:
trainDF = pd.DataFrame()
trainDF['text'] = data["Text"]
trainDF['label'] = data["Text-Type"]

In [3]:
import nltk
sentence_corpus=[]
for sentence in trainDF["text"]:
    sentence_corpus.append(nltk.word_tokenize(sentence))

In [4]:
import gensim
model = gensim.models.Word2Vec(sentence_corpus, min_count=1,vector_size=300,workers=4)

In [5]:
# print(list(model.wv.key_to_index.keys()))

In [6]:
from nltk.tokenize import TweetTokenizer
tw = TweetTokenizer()
sentence_corpus_tweety=[]
for sentence in trainDF["text"]:
    sentence_corpus_tweety.append(tw.tokenize(sentence))
print(sentence_corpus_tweety[0])

['So', 'does', 'that', 'mean', 'it', 'is', 'better', 'than', 'the', 'default', 'one', '?', 'Or', 'what', 'is', 'the', 'standard', 'of', 'comparison', 'among', 'various', 'tokenizers', '?']


In [7]:
model_tweety = gensim.models.Word2Vec(sentence_corpus_tweety, min_count=1,vector_size=300,workers=4)
# print(list(model_tweety.wv.key_to_index.keys()))

In [8]:
#model.wv.similar_by_word("void", 10)
model.wv.similar_by_word("public",10)


[('private', 0.9552631378173828),
 ('static', 0.9546331167221069),
 ('Moon', 0.9500024318695068),
 ('VeeUPage', 0.9471601843833923),
 ('moonName', 0.9446849822998047),
 ('ActionResult', 0.9434309005737305),
 ('bulan', 0.9379481673240662),
 ('getTypeface', 0.9372034072875977),
 ('FragmentActivity', 0.9356179237365723),
 ('planet', 0.9351503252983093)]

In [9]:
model_tweety.wv.similar_by_word("found", 10)

[('asked', 0.9731324911117554),
 ('added', 0.963979959487915),
 ('seems', 0.963098406791687),
 ('made', 0.9619386792182922),
 ('sense', 0.9588219523429871),
 ('knew', 0.9587767124176025),
 ('understood', 0.9572822451591492),
 ('answered', 0.9555019736289978),
 ('ill', 0.9549317955970764),
 ('interested', 0.9547271132469177)]

In [10]:
model.train(sentence_corpus, total_examples=len(sentence_corpus), epochs=50)

(27614591, 38024900)

In [11]:
!pip install tensorflow
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(sentence_corpus)
sequences = tokenizer_obj.texts_to_sequences(sentence_corpus)

In [12]:
print(sequences)

[[86, 177, 25, 1134, 27, 23, 460, 143, 2, 430, 76, 47, 60, 98, 23, 2, 1052, 13, 7090, 533, 1916, 12061, 47], [27, 23, 5731, 9, 15593, 555, 923, 23075, 8, 23076, 1181, 12062, 42, 23077, 99, 2, 23078, 13, 1181, 23079, 5], [777, 27, 9, 15593, 923, 23, 40, 1602, 8, 46, 12063, 5, 17, 44, 211, 2, 3974, 1181, 5732, 185, 14, 27, 585, 579, 24, 98, 17, 41, 496, 1, 13, 502, 1, 87, 1234, 209, 40, 431, 7091, 32, 5733, 1181, 15594, 5], [2, 144, 15, 333, 41, 23080, 3739, 1, 14, 2, 2671, 1181, 1282, 41, 15595, 141, 2, 5732, 5, 108, 2565, 13, 146, 33, 313, 133, 8, 676, 68, 325, 5732, 55, 1002, 706, 3, 1154, 5], [9, 206, 2773, 142, 5253, 14, 1283, 945, 5, 14, 33, 110, 8, 1809, 2, 1959, 643, 12], [279, 2, 1023, 236, 14, 2, 322, 236, 55, 23081, 1727, 5, 2, 414, 17, 238, 2376, 23, 25, 2672, 177, 114, 2295, 24, 2, 1135, 236, 8, 805, 14, 227, 694, 3497, 8, 74, 2, 2205, 570, 5], [17, 352, 2672, 1728, 12, 23082, 1, 23083, 1, 23084, 1, 23085, 14, 23086, 3, 96, 6397, 1728, 4, 96, 1728, 650, 2, 150, 1338, 5, 2, 2

In [13]:
max_length = max([len(s.split()) for s in data["Text"]])
print(max_length)

3046


In [14]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

word_index =tokenizer_obj.word_index
review_pad = pad_sequences(sequences,maxlen=max_length)
print(review_pad.shape)

(23617, 3046)


In [15]:
print(data.shape)

(23617, 5)


In [16]:
filename = 'model.txt'
model.wv.save_word2vec_format(filename,binary=False)

In [17]:
import os
import numpy
embeddings_index = {}
f=open(os.path.join('','model.txt'),encoding='utf-8')
for line in f:
    values=line.split()
    word=values[0]
    coefs=numpy.asarray(values[1:])
    embeddings_index[word]=coefs
f.close()

In [18]:
num_words = len(word_index)+1
EMBEDDING_DIM=300
embedding_matrix=numpy.zeros((num_words,EMBEDDING_DIM))
for word,i in word_index.items():
    if i>num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i]=embedding_vector

In [19]:
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,GRU
# from keras.layers.embeddings import Embedding
from keras.initializers import Constant

model = Sequential()
embedding_layer=Embedding(num_words,EMBEDDING_DIM,embeddings_initializer=Constant(embedding_matrix),input_length=max_length,trainable=False)
model.add(embedding_layer)
model.add(GRU(units=32,dropout=0.2,recurrent_dropout=0.2))
model.add(Dense(3,activation='sigmoid'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [20]:
VALIDATION_SPLIT=0.2
indices=numpy.arange(review_pad.shape[0])
numpy.random.shuffle(indices)
review_pad=review_pad[indices]
y=data["Text-Type"][indices]
num_validation_samples=int(VALIDATION_SPLIT*review_pad.shape[0])
X_train=review_pad[:-num_validation_samples]
y_train=y[:-num_validation_samples]
X_test=review_pad[-num_validation_samples:]
y_test=y[-num_validation_samples:]

from keras.utils import to_categorical
y_binary_train = to_categorical(y_train)
y_binary_test = to_categorical(y_test)

In [ ]:
model.fit(X_train,y_binary_train,batch_size=128,epochs=25,validation_data=(X_test,y_binary_test),verbose=2)

Epoch 1/25
